In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, ConfusionMatrixDisplay
import numpy as np
from sklearn.model_selection import train_test_split, KFold


In [8]:
df = pd.read_csv('df_att.csv')
X = df.drop(['Heart Disease', 'Unnamed: 0'], axis = 1)
y = df['Heart Disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits = 5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X_train))
cat_vars = list(X_train.select_dtypes(exclude='number').columns)

for train_idx, val_idx in kf.split(X_train):
    X_tr = X_train.iloc[train_idx]
    X_val = X_train.iloc[val_idx]
    y_tr = y_train.iloc[train_idx]
    y_val = y_train.iloc[train_idx]
    
    # Definindo o modelo
    model  =  CatBoostClassifier(iterations=500, 
                                 cat_features=cat_vars,
                                 learning_rate=0.05,
                                 depth=6,
                                 loss_function='Logloss',
                                 eval_metric='AUC', 
                                 verbose=100)

    model.fit(X_tr, y_tr)

    oof_preds[val_idx] = model.predict_proba(X_val)[:,1]


oof_preds.shape, X_train.shape

0:	total: 315ms	remaining: 2m 37s
100:	total: 18.1s	remaining: 1m 11s
200:	total: 35.9s	remaining: 53.4s
300:	total: 54.2s	remaining: 35.8s
400:	total: 1m 13s	remaining: 18s
499:	total: 1m 31s	remaining: 0us
0:	total: 200ms	remaining: 1m 39s
100:	total: 18.2s	remaining: 1m 12s
200:	total: 36.9s	remaining: 54.9s
300:	total: 55.1s	remaining: 36.4s
400:	total: 1m 14s	remaining: 18.4s
499:	total: 1m 33s	remaining: 0us
0:	total: 201ms	remaining: 1m 40s
100:	total: 18.6s	remaining: 1m 13s
200:	total: 37.2s	remaining: 55.3s
300:	total: 56.9s	remaining: 37.6s
400:	total: 1m 16s	remaining: 18.8s
499:	total: 1m 36s	remaining: 0us
0:	total: 206ms	remaining: 1m 42s
100:	total: 18.9s	remaining: 1m 14s
200:	total: 38s	remaining: 56.5s
300:	total: 57.1s	remaining: 37.7s
400:	total: 1m 15s	remaining: 18.7s
499:	total: 1m 34s	remaining: 0us
0:	total: 202ms	remaining: 1m 40s
100:	total: 18.5s	remaining: 1m 13s
200:	total: 37.4s	remaining: 55.7s
300:	total: 56.2s	remaining: 37.1s
400:	total: 1m 15s	remai

((504000,), (504000, 13))

In [10]:
oof_preds_csv = pd.DataFrame({'oof_preds': oof_preds}).to_csv('oof_preds.csv', index=False)